## 1. Mount your Google Drive and Install Required Libraries

Mounting your google drive to google colab is helpful to avoid loosing data if a process in google colab crashes, when the process completes it saves the output and you can check on it whenever.

Install Roboflow and Ultralytics (YoloV8), at the time of writing this file roboflow 1.1.9 is the latest version which requires ultralytics 8.0.196.

In [ ]:
from google.colab import drive
from os import makedirs

drive.mount("/content/drive")
makedirs("/content/drive/MyDrive/Colab/object_detection/yolov8", exist_ok=True)
%cd "/content/drive/MyDrive/Colab/object_detection/yolov8"
%pip install roboflow==1.1.9
%pip install ultralytics==8.0.196 # Roboflow is dependant on version 8.0.196
%pip install openvino==2023.1.0 openvino-dev==2023.1.0 onnx==1.15.0

## 2. Load your Dataset
After following the steps in Preparing the Dataset, copy the Jupyter code snippet from the export function excluding the pip install part, overwritting the example below then run it.

In [ ]:
from roboflow import Roboflow
%cd "/content/drive/MyDrive/Colab/object_detection/yolov8"
rf = Roboflow(api_key="<key>")
project = rf.workspace("<workspace>").project("<project>")
dataset = project.version(1).download("yolov8")

## 3. Train the Model
Using the roboflow dataset train the model.

In [ ]:
from datetime import datetime
from ultralytics import YOLO

%cd "/content/drive/MyDrive/Colab/object_detection/yolov8"
project = "handgun_detection"
name = f"run_{datetime.now().strftime('%Y-%m-%d_%H-%M')}"
model = YOLO("yolov8n.pt") # Using the smallest pretrained dataset to avoid overfitting of a small dataset.
results = model.train(data=f"{dataset.location}/data.yaml", epochs=200, imgsz=480, project=project, name=name)

## 4. View Training Summary

In [ ]:
from IPython import display
display.Image(f"{project}/{name}/results.png")

## 5. Apply Model to the Test Dataset
After running the below cell you should find the result images on your drive at "Colab/object_detection/yolov8/runs/detect/{name of this run}"

In [ ]:
%cd "/content/drive/MyDrive/Colab/object_detection/yolov8"
model = f"{project}/{name}/weights/best.pt"
!yolo task=detect mode=predict model= $model source= {dataset.location}/test/images imgsz= 480 name= $name

## 6. Export the Model
After running the below cell you can download the models from "Colab/object_detection/yolov8/handgun_detection/{name of this run}/weights/best_openvino_model"

In [ ]:
from ultralytics import YOLO

model = f"{project}/{name}/weights/best.pt"
model_path = f"{project}/{name}/weights/best.pt"
model = YOLO(model_path)
model.export(format="openvino", dynamic=True)